In [1]:
import tensorflow as tf
import numpy as np
import glob
import cv2


class FaceDetect:
    def __init__(self, images, labels):
        #print("Initialized")
        self.images = images
        self.labels = labels
        self.epochs_completed = 0
        self.index_in_epoch = 0
        #print(self.images.shape)
        #print(self.labels.shape)

    #For next batch
    def next_batch(self, batch_size):
        """
        input-batch size
        returns batch_x,batch_y which contain the training and test data of this batch
        batch_x-[batch_size patch_size*patch_size*3]
        batch_y-labels
        index_in_epoch
        """
        #print("Next batch called")
        start = self.index_in_epoch
        self.index_in_epoch += batch_size
        #print(self.index_in_epoch)
        if self.index_in_epoch > self.images.shape[0]:
            #print("Shuffling")
            # Shuffle the data
            perm = np.arange(self.images.shape[0])
            np.random.shuffle(perm)
            self.images = self.images[perm]
            self.labels = self.labels[perm]
            # Start next epoch
            start = 0
            self.index_in_epoch = batch_size
            assert batch_size <= self.images.shape[0]
        end = self.index_in_epoch
        return self.images[start:end], self.labels[start:end]


#import data
patch_size = 64
num_training_images = 10000
num_testing_images = 1000

train_face_data = np.zeros((num_training_images, patch_size*patch_size*3))
train_nonface_data = np.zeros((num_training_images, patch_size*patch_size*3))
labels_face_train = np.ones((1, num_training_images))  # 1*50
labels_nonface_train = np.zeros((1, num_training_images))  # 1*50

labels_face_test = np.ones((1, num_testing_images))  # 1*50
labels_nonface_test = np.zeros((1, num_testing_images))  # 1*50

test_face_data = np.zeros((num_testing_images, patch_size*patch_size*3))
test_nonface_data = np.zeros((num_testing_images, patch_size*patch_size*3))

index = 0
k = 0
for img in glob.glob("Dataset//extracted_faces//*.jpg"):
    if(index == num_testing_images and k == 1):
        break
    if(index == num_training_images and k == 0):
        print("Training data loaded for face")
        k = 1
        index = 0
    else:
        n = cv2.imread(img)  # Read the face
        if(n.size and k == 0):
            train_face_data[index] = n.flatten()
            index = index+1
        elif(n.size and k == 1):
            test_face_data[index] = n.flatten()
            index = index+1

print("Training Data Face", train_face_data.shape)
print("Testing Data Face", test_face_data.shape)

index = 0
k = 0
for img in glob.glob("Dataset//extracted_nonfaces//*.jpg"):
    if(index == num_testing_images and k == 1):
        break
    if(index == num_training_images and k == 0):
        print("Training data loaded for nonface")
        k = 1
        index = 0
    else:
        n = cv2.imread(img)  # Read the face
        if(n.size and k == 0):
            train_nonface_data[index] = n.flatten()
            index = index+1
        elif(n.size and k == 1):
            test_nonface_data[index] = n.flatten()
            index = index+1

print("Training Data NonFace", train_nonface_data.shape)
print("Testing Data NonFace", test_nonface_data.shape)

train_all = np.vstack((train_face_data, train_nonface_data))
test_all = np.vstack((test_face_data, test_nonface_data))

labels_l = np.hstack((labels_face_train, labels_nonface_train))
labels_r = np.hstack((labels_nonface_train, labels_face_train))
labels_all = np.vstack((labels_l, labels_r)).T

labels_l = np.hstack((labels_face_test, labels_nonface_test))
labels_r = np.hstack((labels_nonface_test, labels_face_test))
labels_all_test = np.vstack((labels_l, labels_r)).T

#normalizing
train_all = (train_all-np.mean(train_all))/np.var(train_all)
test_all = test_all-np.mean(test_all)/np.var(test_all)
print(train_all.shape)
print(labels_all.shape)
#print(labels_all)

model = FaceDetect(train_all, labels_all)


InvalidArgumentError: Input to reshape is a tensor with 2097152 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_1 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](MaxPool_1-1-0-TransposeNCHWToNHWC-LayoutOptimizer, Reshape_1/shape)]]

Caused by op 'Reshape_1', defined at:
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-pack

In [2]:

# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = patch_size*patch_size*3 # data input (img shape: patch_size*patch_size)
num_classes = 2  # total classes
dropout = 0.75  # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input]) #batch_size, num_input
Y = tf.placeholder(tf.float32, [None, num_classes]) #batch_size, num_classes
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, patch_size, patch_size, 3]) #we have 3 channels, so modify

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


InvalidArgumentError: Input to reshape is a tensor with 2097152 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_1 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](MaxPool_1-1-0-TransposeNCHWToNHWC-LayoutOptimizer, Reshape_1/shape)]]

Caused by op 'Reshape_1', defined at:
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-pack

In [3]:

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = model.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x,
                                      Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " +
                  "{:.4f}".format(loss) + ", Training Accuracy= " +
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    """
    # Calculate accuracy for test images
    print("Testing Accuracy:",
          sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                        Y: mnist.test.labels[:256],
                                        keep_prob: 1.0}))
    """


InvalidArgumentError: Input to reshape is a tensor with 2097152 values, but the requested shape requires a multiple of 3136
	 [[Node: Reshape_1 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](MaxPool_1-1-0-TransposeNCHWToNHWC-LayoutOptimizer, Reshape_1/shape)]]

Caused by op 'Reshape_1', defined at:
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/hariharan/anaconda3/envs/DeepLearning/lib/python3.6/site-pack